# Preprocesamiento

## Lectura de datos

In [41]:
import pandas as pd
import nltk
import gensim

df_true = pd.read_csv("./dataset/True.csv")
df_fake = pd.read_csv("./dataset/Fake.csv")

#### Agregar columna de identificación de Fake News

In [42]:
df_true['isfake'] = 1

df_true.head()

,title,text,subject,date,isfake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [43]:
df_fake['isfake'] = 0

df_fake.head()

,title,text,subject,date,isfake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [61]:
df = pd.concat([df_true, df_fake]).reset_index(drop = True)

### Combinación de título y texto

In [45]:
df['original'] = df['title'] + '. ' + df['text']
df['original'].replace("..", ".")
df.head()

,title,text,subject,date,isfake,original
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1,"As U.S. budget fight looms, Republicans flip t..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1,U.S. military to accept transgender recruits o...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1,Senior U.S. Republican senator: 'Let Mr. Muell...
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1,FBI Russia probe helped by Australian diplomat...
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1,Trump wants Postal Service to charge 'much mor...


#### Muestra de la combinación anterior

In [46]:
df['title'][0]

'As U.S. budget fight looms, Republicans flip their fiscal script'

In [47]:
df['text'][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [48]:
df['original'][0]

'As U.S. budget fight looms, Republicans flip their fiscal script. WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for no

In [49]:
import os

if os.path.exists(r'dataset/lemmatized.csv'):
    df = pd.read_csv(r'dataset/lemmatized.csv')
else:
    import stanza, torch, gc
    from stanza.models.common.doc import Document
    nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

    def token_lemma(_text):
        tokend = []
        tokens_lemmatized = []
        stanza_doc = stanza.Document([], text=_text)
        stanza_doc = nlp(stanza_doc)
        for sent in stanza_doc.sentences:
            for word in sent.words:
                tokens_lemmatized.append(word.lemma)
        return tokens_lemmatized
    
    df['lemma'] = df['original'].apply(token_lemma)
        
    df.to_csv(r'dataset/lemmatized.csv')

In [50]:
df.head()

,Unnamed: 0,title,text,subject,date,isfake,original,lemma
0,0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1,"As U.S. budget fight looms, Republicans flip t...","['as', 'U.S.', 'budget', 'fight', 'loom', ',',..."
1,1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1,U.S. military to accept transgender recruits o...,"['U.S.', 'military', 'to', 'accept', 'transgen..."
2,2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1,Senior U.S. Republican senator: 'Let Mr. Muell...,"['senior', 'U.S.', 'republican', 'senator', ':..."
3,3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1,FBI Russia probe helped by Australian diplomat...,"['FBI', 'Russia', 'probe', 'help', 'by', 'Aust..."
4,4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1,Trump wants Postal Service to charge 'much mor...,"['Trump', 'want', 'postal', 'Service', 'to', '..."


----

### Descargar Stop-Words

In [51]:
# nltk.download("stopwords")

In [52]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stop_words.extend(list(gensim.parsing.preprocessing.STOPWORDS))
stop_words = list(set(stop_words))


In [53]:
for word in stop_words:
    print(word, end=', ')

toward, became, full, didn't, con, give, what, s, and, thick, yours, once, been, mine, thin, otherwise, moreover, did, empty, whom, to, side, keep, throughout, weren't, when, should, show, then, others, some, system, why, o, couldnt, on, whereafter, really, you, twelve, everyone, perhaps, her, own, co, nothing, due, she, ain, will, someone, move, against, hence, eight, never, everything, four, take, becomes, along, a, every, isn, since, any, hasnt, under, least, all, might, its, using, still, mustn, interest, wouldn't, were, however, within, ll, eleven, either, found, again, into, than, mightn, where, whereupon, it's, nor, being, wherever, neither, cry, more, me, please, herself, nobody, off, without, must, everywhere, also, so, always, sometime, wasn't, forty, does, rather, before, d, above, now, until, ltd, over, ie, your, sincere, was, hereby, an, are, say, whatever, regarding, most, them, km, down, doesn, after, themselves, eg, because, aren't, per, with, fire, thereby, somewhere, 

### Eliminar Stop-Words del dataset

In [54]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if (len(token) > 3 and token not in stop_words) or '.' in token or ',' in token:
            result.append(token)
            
    return result

In [55]:
df['clean'] = df['lemma'].apply(preprocess)
df.head()

,Unnamed: 0,title,text,subject,date,isfake,original,lemma,clean
0,0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1,"As U.S. budget fight looms, Republicans flip t...","['as', 'U.S.', 'budget', 'fight', 'loom', ',',...","[budget, fight, loom, republicans, flip, fisca..."
1,1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1,U.S. military to accept transgender recruits o...,"['U.S.', 'military', 'to', 'accept', 'transgen...","[military, accept, transgender, recruit, monda..."
2,2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1,Senior U.S. Republican senator: 'Let Mr. Muell...,"['senior', 'U.S.', 'republican', 'senator', ':...","[senior, republican, senator, mueller, washing..."
3,3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1,FBI Russia probe helped by Australian diplomat...,"['FBI', 'Russia', 'probe', 'help', 'by', 'Aust...","[russia, probe, help, australian, diplomat, wa..."
4,4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1,Trump wants Postal Service to charge 'much mor...,"['Trump', 'want', 'postal', 'Service', 'to', '...","[trump, want, postal, service, charge, amazon,..."


### Ejemplo de registro después de remover Stop-Words

In [56]:
print(df['clean'][0])

['budget', 'fight', 'loom', 'republicans', 'flip', 'fiscal', 'script', 'washington', 'reuters', 'head', 'conservative', 'republican', 'faction', 'congress', 'vote', 'month', 'huge', 'expansion', 'national', 'debt', 'fiscal', 'conservative', 'sunday', 'urge', 'budget', 'restraint', 'sharp', 'pivot', 'republicans', 'representative', 'mark', 'meadows', 'speak', 'face', 'nation', 'draw', 'hard', 'line', 'federal', 'spending', 'lawmaker', 'brace', 'battle', 'january', 'return', 'holiday', 'wednesday', 'lawmaker', 'begin', 'pass', 'federal', 'budget', 'fight', 'likely', 'link', 'issue', 'immigration', 'policy', 'november', 'congressional', 'election', 'campaign', 'approach', 'republicans', 'seek', 'control', 'congress', 'president', 'donald', 'trump', 'republicans', 'want', 'budget', 'increase', 'military', 'spending', 'democrats', 'want', 'proportional', 'increase', 'defense', 'discretionary', 'spend', 'program', 'support', 'education', 'scientific', 'research', 'infrastructure', 'public', 

### Total de palabras en el Dataset

In [57]:
list_of_words = []
unique_words = set()
for document in df.clean:
    for word in document:
        list_of_words.append(word)
        unique_words.add(word)
        
total_words = len(list_of_words)  # total words
unique_words = len(unique_words)   # total unique words
print("Total words:" + str(total_words) + " unique_words:" + str(unique_words))

Total words:8845109 unique_words:96149


Unir palabras nuevamente 

In [58]:
df['clean_joined'] = df['clean'].apply(lambda x: " ".join(x))

df.head()

,Unnamed: 0,title,text,subject,date,isfake,original,lemma,clean,clean_joined
0,0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1,"As U.S. budget fight looms, Republicans flip t...","['as', 'U.S.', 'budget', 'fight', 'loom', ',',...","[budget, fight, loom, republicans, flip, fisca...",budget fight loom republicans flip fiscal scri...
1,1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1,U.S. military to accept transgender recruits o...,"['U.S.', 'military', 'to', 'accept', 'transgen...","[military, accept, transgender, recruit, monda...",military accept transgender recruit monday pen...
2,2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1,Senior U.S. Republican senator: 'Let Mr. Muell...,"['senior', 'U.S.', 'republican', 'senator', ':...","[senior, republican, senator, mueller, washing...",senior republican senator mueller washington r...
3,3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1,FBI Russia probe helped by Australian diplomat...,"['FBI', 'Russia', 'probe', 'help', 'by', 'Aust...","[russia, probe, help, australian, diplomat, wa...",russia probe help australian diplomat washingt...
4,4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1,Trump wants Postal Service to charge 'much mor...,"['Trump', 'want', 'postal', 'Service', 'to', '...","[trump, want, postal, service, charge, amazon,...",trump want postal service charge amazon shipme...


### Ejemplo de texto limpio

In [59]:
df['clean_joined'][0]

'budget fight loom republicans flip fiscal script washington reuters head conservative republican faction congress vote month huge expansion national debt fiscal conservative sunday urge budget restraint sharp pivot republicans representative mark meadows speak face nation draw hard line federal spending lawmaker brace battle january return holiday wednesday lawmaker begin pass federal budget fight likely link issue immigration policy november congressional election campaign approach republicans seek control congress president donald trump republicans want budget increase military spending democrats want proportional increase defense discretionary spend program support education scientific research infrastructure public health environmental protection trump administration willing increase defense discretionary spending percent meadows chairman small influential house freedom caucus program democrats need government raise percent fiscal conservative rationale eventually people money mea

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    44898 non-null  int64 
 1   title         44898 non-null  object
 2   text          44898 non-null  object
 3   subject       44898 non-null  object
 4   date          44898 non-null  object
 5   isfake        44898 non-null  int64 
 6   original      44898 non-null  object
 7   lemma         44898 non-null  object
 8   clean         44898 non-null  object
 9   clean_joined  44898 non-null  object
dtypes: int64(2), object(8)
memory usage: 3.4+ MB


### Se eliminarán las siguientes filas que tienen valores erroneos

In [20]:
df_to_delete = df[~df["date"].str[-2:-1].str.isdigit()]
df_to_delete

,Unnamed: 0,title,text,subject,date,isfake,original,lemma,clean,clean_joined
30775,30775,https://100percentfedup.com/served-roy-moore-v...,https://100percentfedup.com/served-roy-moore-v...,politics,https://100percentfedup.com/served-roy-moore-v...,0,https://100percentfedup.com/served-roy-moore-v...,['https://100percentfedup.com/served-roy-moore...,"[https, percentfedup, served, moore, vietnamle...",https percentfedup served moore vietnamletter ...
36924,36924,https://100percentfedup.com/video-hillary-aske...,https://100percentfedup.com/video-hillary-aske...,politics,https://100percentfedup.com/video-hillary-aske...,0,https://100percentfedup.com/video-hillary-aske...,['https://100percentfedup.com/video-hillary-as...,"[https, percentfedup, video, hillary, asked, t...",https percentfedup video hillary asked trump w...
36925,36925,https://100percentfedup.com/12-yr-old-black-co...,https://100percentfedup.com/12-yr-old-black-co...,politics,https://100percentfedup.com/12-yr-old-black-co...,0,https://100percentfedup.com/12-yr-old-black-co...,['https://100percentfedup.com/12-yr-old-black-...,"[https, percentfedup, black, conservative, vid...",https percentfedup black conservative video ob...
37256,37256,https://fedup.wpengine.com/wp-content/uploads/...,https://fedup.wpengine.com/wp-content/uploads/...,politics,https://fedup.wpengine.com/wp-content/uploads/...,0,https://fedup.wpengine.com/wp-content/uploads/...,['https://fedup.wpengine.com/wp-content/upload...,"[https, fedup, wpengine, content, uploads, htt...",https fedup wpengine content uploads https fed...
37257,37257,https://fedup.wpengine.com/wp-content/uploads/...,https://fedup.wpengine.com/wp-content/uploads/...,politics,https://fedup.wpengine.com/wp-content/uploads/...,0,https://fedup.wpengine.com/wp-content/uploads/...,['https://fedup.wpengine.com/wp-content/upload...,"[https, fedup, wpengine, content, uploads, ent...",https fedup wpengine content uploads entitled ...
38849,38849,https://fedup.wpengine.com/wp-content/uploads/...,https://fedup.wpengine.com/wp-content/uploads/...,Government News,https://fedup.wpengine.com/wp-content/uploads/...,0,https://fedup.wpengine.com/wp-content/uploads/...,['https://fedup.wpengine.com/wp-content/upload...,"[https, fedup, wpengine, content, uploads, htt...",https fedup wpengine content uploads https fed...
38850,38850,https://fedup.wpengine.com/wp-content/uploads/...,https://fedup.wpengine.com/wp-content/uploads/...,Government News,https://fedup.wpengine.com/wp-content/uploads/...,0,https://fedup.wpengine.com/wp-content/uploads/...,['https://fedup.wpengine.com/wp-content/upload...,"[https, fedup, wpengine, content, uploads, ent...",https fedup wpengine content uploads entitled ...
40350,40350,Homepage,[vc_row][vc_column width= 1/1 ][td_block_trend...,left-news,MSNBC HOST Rudely Assumes Steel Worker Would N...,0,Homepage. [vc_row][vc_column width= 1/1 ][td_b...,"['homepage', '.', '[', 'vc_row', ']', '[', 'vc...","[homepage, vc_row, vc_column, width, limit, so...",homepage vc_row vc_column width limit sort fea...
43286,43286,https://fedup.wpengine.com/wp-content/uploads/...,https://fedup.wpengine.com/wp-content/uploads/...,left-news,https://fedup.wpengine.com/wp-content/uploads/...,0,https://fedup.wpengine.com/wp-content/uploads/...,['https://fedup.wpengine.com/wp-content/upload...,"[https, fedup, wpengine, content, uploads, htt...",https fedup wpengine content uploads https fed...
43287,43287,https://fedup.wpengine.com/wp-content/uploads/...,https://fedup.wpengine.com/wp-content/uploads/...,left-news,https://fedup.wpengine.com/wp-content/uploads/...,0,https://fedup.wpengine.com/wp-content/uploads/...,['https://fedup.wpengine.com/wp-content/upload...,"[https, fedup, wpengine, content, uploads, ent...",https fedup wpengine content uploads entitled ...


In [21]:
df = df.drop(df_to_delete.index.tolist())

### Borrar las columnas que no se usarán

In [40]:
df = df.drop(columns=["Unnamed: 0", "lemma", "clean", "original"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44888 entries, 0 to 44897
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         44888 non-null  object
 1   text          44888 non-null  object
 2   subject_id    44888 non-null  int64 
 3   date          44888 non-null  object
 4   isfake        44888 non-null  int64 
 5   clean_joined  44888 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.4+ MB


## Guardar los datos obtenidos y cambiados en SQLite

### Archivo donde se va guardar la base de datos SQLite

In [23]:
sqlite_db = "dataset/fake_news1.sqlite"

### Función para ejecutar código SQL en SQLite

In [24]:
import sqlite3

def execute_sql(query, path_db):
    conn = sqlite3.connect(path_db)
    cursor = conn.cursor()
    cursor.execute(query)
    conn.commit()
    conn.close()

### La columna "Subject" se puede dejar como una llave foránea a otra tabla que contenga el valor actual. En la siguiente celda, se cambiara la columna por un identificador:

In [25]:
subject_dict = {}
id_subject = 0
for subject in set(df["subject"]):
    subject_dict[subject] = id_subject
    id_subject += 1

df["subject"] = df["subject"].apply(lambda x: subject_dict[x])
df = df.rename(columns={'subject':'subject_id'})

df.head()

,title,text,subject_id,date,isfake,clean_joined
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1,"December 31, 2017",1,budget fight loom republicans flip fiscal scri...
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1,"December 29, 2017",1,military accept transgender recruit monday pen...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1,"December 31, 2017",1,senior republican senator mueller washington r...
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1,"December 30, 2017",1,russia probe help australian diplomat washingt...
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1,"December 29, 2017",1,trump want postal service charge amazon shipme...


### Ahora se van a crear las primeras 2 tablas:

In [26]:

sql_subject = """CREATE TABLE SUBJECT (
    subject_id    INTEGER PRIMARY KEY,
    subject       TEXT
);"""

sql_news = """CREATE TABLE NEWS (
    news_id       INTEGER PRIMARY KEY,
    title         TEXT,
    text          TEXT,
    subject_id    INTEGER,
    date          TEXT,
    isfake        INTEGER,
    clean_joined  TEXT,
    FOREIGN KEY (subject_id) REFERENCES SUBJECT (subject_id) 
);"""

execute_sql("DROP TABLE IF EXISTS SUBJECT", sqlite_db)
execute_sql(sql_subject, sqlite_db)
execute_sql("DROP TABLE IF EXISTS NEWS", sqlite_db)
execute_sql(sql_news, sqlite_db)

### Insertar el dataframe "df" en la tabla "NEWS" y obtener el dataframe de esa tabla:

In [27]:
conn = sqlite3.connect(sqlite_db)
df.to_sql("NEWS", conn, if_exists="append", index=False)
df_news = pd.read_sql_query("SELECT * FROM NEWS", conn)
conn.close()

df_news.head()

,news_id,title,text,subject_id,date,isfake,clean_joined
0,1,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1,"December 31, 2017",1,budget fight loom republicans flip fiscal scri...
1,2,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1,"December 29, 2017",1,military accept transgender recruit monday pen...
2,3,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1,"December 31, 2017",1,senior republican senator mueller washington r...
3,4,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1,"December 30, 2017",1,russia probe help australian diplomat washingt...
4,5,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1,"December 29, 2017",1,trump want postal service charge amazon shipme...


### Mediante los valores extraídos anteriormente de Subject, se crea un dataframe que se inserta en la tabla "SUBJECT" y se obtiene el dataframe correspondiente a esta tabla:

In [28]:
df_subject = pd.DataFrame({"subject_id": subject_dict.values(), "subject": subject_dict.keys()})

conn = sqlite3.connect(sqlite_db)
df_subject.to_sql("SUBJECT", conn, if_exists="append", index=False)
df_subject = pd.read_sql_query("SELECT * FROM SUBJECT", conn)
conn.close()

df_subject.head()

,subject_id,subject
0,0,Government News
1,1,politicsNews
2,2,left-news
3,3,Middle-east
4,4,worldnews


### Obtener un dataframe de la base de datos únicamente con los campos "Título+texto" y "id" para extraer Entidades y Sentimientos mediante Stanza:

In [29]:

conn = sqlite3.connect(sqlite_db)
df_sql = pd.read_sql_query("SELECT news_id, title || '. ' || text as original FROM NEWS", conn)
conn.close()

df_sql.head()

,news_id,original
0,1,"As U.S. budget fight looms, Republicans flip t..."
1,2,U.S. military to accept transgender recruits o...
2,3,Senior U.S. Republican senator: 'Let Mr. Muell...
3,4,FBI Russia probe helped by Australian diplomat...
4,5,Trump wants Postal Service to charge 'much mor...


### Extracción de sentimientos y entidades mediante Stanza y preparación de datos para insersión en la base de datos:

In [31]:
import stanza, torch, gc
from stanza.models.common.doc import Document
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner,sentiment')

sentence_list = []
sentiment_list = []
entity_sentence_dict = {}
doc_id_list = []


def ner(news_id, original):
    global sentence_list
    global sentiment_list
    global entity_sentence_dict
    global doc_id_list
    stanza_doc = stanza.Document([], text=original)
    stanza_doc = nlp(stanza_doc)
    for sent in stanza_doc.sentences:
        sentence_list.append(sent.text)
        sentence_id = len(sentence_list) - 1
        doc_id_list.append(news_id)
        sentiment_list.append(sent.sentiment)
        for ent in sent.ents:
            if ent.text + "^" + ent.type not in entity_sentence_dict:
                entity_sentence_dict[ent.text + "^" + ent.type] = set()
                
            entity_sentence_dict[ent.text + "^" + ent.type].add(sentence_id)
        
df_sql.apply(lambda x: ner(x.news_id, x.original), axis=1)


2021-07-22 21:05:14 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-07-22 21:05:14 INFO: Use device: gpu
2021-07-22 21:05:14 INFO: Loading: tokenize
2021-07-22 21:05:14 INFO: Loading: sentiment
2021-07-22 21:05:14 INFO: Loading: ner
2021-07-22 21:05:14 INFO: Done loading processors!


0        None
1        None
2        None
3        None
4        None
         ... 
44883    None
44884    None
44885    None
44886    None
44887    None
Length: 44888, dtype: object

### Para guardar las entidades encontradas, se crearán las tablas "TYPE_ENTITY" para guardar el tipo de entidad y "ENTITY" para guardar la entidad en si:

In [32]:

sql_type_entity = """CREATE TABLE TYPE_ENTITY (
    type_id       INTEGER PRIMARY KEY,
    type          TEXT
);"""

sql_entity = """CREATE TABLE ENTITY (
    ent_id        INTEGER PRIMARY KEY,
    ent_text      TEXT,
    type_id       INTEGER,
    FOREIGN KEY (type_id) REFERENCES TYPE_ENTITY (type_id) 
);"""

execute_sql("DROP TABLE IF EXISTS TYPE_ENTITY", sqlite_db)
execute_sql(sql_type_entity, sqlite_db)
execute_sql("DROP TABLE IF EXISTS ENTITY", sqlite_db)
execute_sql(sql_entity, sqlite_db)

### Extraer los datos para tener los tipos de entidades en la tabla "TYPE_ENTITY"

In [33]:

entity_list = []
type_entity_list = []
type_dict = {}
type_id = 0

type_ent_list = []
for key in entity_sentence_dict:
    ent, typ = key.split("^")
    if typ not in type_dict:
        type_dict[typ] = type_id
        type_id += 1
    entity_list.append(ent)
    type_entity_list.append(type_dict[typ])

### Insertar los valores de tipos de entidades en la tabla "TYPE_ENTITY":

In [34]:
df_type = pd.DataFrame({"type_id": type_dict.values(), "type": type_dict.keys()})
conn = sqlite3.connect(sqlite_db)
df_type.to_sql("TYPE_ENTITY", conn, if_exists="append", index=False)
df_type = pd.read_sql_query("SELECT * FROM TYPE_ENTITY", conn)
conn.close()

df_type

,type_id,type
0,0,GPE
1,1,NORP
2,2,ORG
3,3,DATE
4,4,PERSON
5,5,PERCENT
6,6,MONEY
7,7,CARDINAL
8,8,ORDINAL
9,9,LAW


### Insertar los valores de entidades y los valores de la llave foránea en la tabla "ENTITY":

In [35]:
df_entity = pd.DataFrame({"ent_id": [*range(0,len(entity_list))], "ent_text": entity_list, "type_id": type_entity_list})

conn = sqlite3.connect(sqlite_db)
df_entity.to_sql("ENTITY", conn, if_exists="append", index=False)
df_entity = pd.read_sql_query("SELECT * FROM ENTITY", conn)
conn.close()

df_entity

,ent_id,ent_text,type_id
0,0,U.S.,0
1,1,Republicans,1
2,2,WASHINGTON,0
3,3,Reuters,2
4,4,Republican,1
...,...,...,...
195575,195575,Rahm Failed Us,4
195576,195576,Emanuel,2
195577,195577,Dick Darth Vader Cheney,4
195578,195578,19 years old,3


### Creación de las tablas "SENTENCE" para guardar las oraciones extraídas y "SENTENCE_ENTITY" para guardar la relación de las entidades con las oraciones:

In [36]:
sql_sent = """CREATE TABLE SENTENCE (
    sent_id        INTEGER PRIMARY KEY,
    news_id         INTEGER,
    sent_text      TEXT,
    sentiment      INTEGER,
    FOREIGN KEY (news_id) REFERENCES NEWS (news_id)
);"""

sql_sent_ent = """CREATE TABLE SENTENCE_ENTITY (
    sent_id        INTEGER,
    ent_id         INTEGER,
    FOREIGN KEY (sent_id) REFERENCES SENTENCE (sent_id),
    FOREIGN KEY (ent_id) REFERENCES ENTITY (ent_id) 
);"""


execute_sql("DROP TABLE IF EXISTS SENTENCE", sqlite_db)
execute_sql(sql_sent, sqlite_db)
execute_sql("DROP TABLE IF EXISTS SENTENCE_ENTITY", sqlite_db)
execute_sql(sql_sent_ent, sqlite_db)

### Preparación de los datos para insertar en las tablas "SENTENCE" y "SENTENCE_ENTITY"

In [37]:

id_entity = []
index = 0
id_sentence = []
for key in entity_sentence_dict: 
    for sent_id in entity_sentence_dict[key]:
        id_entity.append(index)
        id_sentence.append(sent_id)
    index += 1


### Insertar los datos correspondientes a la tabla "SENTENCE_ENTITY"

In [38]:
df_sent_ent = pd.DataFrame({"ent_id": id_entity, "sent_id": id_sentence})

conn = sqlite3.connect(sqlite_db)
df_sent_ent.to_sql("SENTENCE_ENTITY", conn, if_exists="append", index=False)
df_sent_ent = pd.read_sql_query("SELECT * FROM SENTENCE_ENTITY", conn)
conn.close()

df_sent_ent

,sent_id,ent_id
0,0,0
1,1,0
2,2,0
3,131073,0
4,131082,0
...,...,...
2148850,889135,195575
2148851,889142,195576
2148852,889147,195577
2148853,889151,195578


### Insertar los datos correspondientes a la tabla "SENTENCE"

In [39]:
df_sent = pd.DataFrame({"sent_id": [*range(0,len(sentence_list))], "news_id": doc_id_list, "sent_text": sentence_list, "sentiment": sentiment_list})

conn = sqlite3.connect(sqlite_db)
df_sent.to_sql("SENTENCE", conn, if_exists="append", index=False)
df_sent = pd.read_sql_query("SELECT * FROM SENTENCE", conn)
conn.close()

df_sent

,sent_id,news_id,sent_text,sentiment
0,0,1,"As U.S. budget fight looms, Republicans flip t...",0
1,1,1,WASHINGTON (Reuters) - The head of a conservat...,1
2,2,1,In keeping with a sharp pivot under way among ...,1
3,3,1,"“Face the Nation,” drew a hard line on federal...",1
4,4,1,When they return from the holidays on Wednesda...,0
...,...,...,...,...
914924,914924,44888,"Not surprisingly, at the end of his prearrange...",0
914925,914925,44888,Of all the U.S. Officials CNN could have broug...,1
914926,914926,44888,"Clearly, this looks like a neocon stunt.",1
914927,914927,44888,Stay tuned for more updates.,1


## Para este proyecto este va a ser todo el preprocesamiento que se va a realizar, para ver los detalles de implementación de la base de datos y la exploración de los datos, revisar el notebook [exploration.ipynb](https://github.com/fake-news-unal/fake-news/blob/main/exploration.ipynb)